# pytorch setup, imports and constants initialization

In [0]:
# !pip uninstall tensorflow --yes
# !pip uninstall tensorboard --yes
# !pip install --ignore-installed tf-nightly

In [1]:
import os
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torch torchvision
  
import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

     |████████████████████████████████| 614.8MB 29kB/s 
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.0.1 which is incompatible.
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
cuda:0


In [2]:
!pip3 install pymagnitude
from pymagnitude import *

import time
import argparse
import numpy as np
import time
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from sklearn.metrics import f1_score
from pdb import set_trace as debug
import pickle


PAD = 0
UNK = 1
BOS = 2
EOS = 3

PAD_WORD = '<blank>'
UNK_WORD = '<unk>'
BOS_WORD = '<s>'
EOS_WORD = '</s>'

CAT = ['PER', 'ORG', 'LOC', 'MISC']
POSITION = ['I', 'B']
LABEL_INDEX = [PAD_WORD] + ['O'] + ["{}-{}".format(position, cat) for cat in CAT for position in POSITION]

     |████████████████████████████████| 5.4MB 4.0MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918206 sha256=2eebdfccd9b6d96ef7c6755f8408b0e320ae300cd4f19896b2b4bf472355710b
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude


# Connect to Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd "/content/gdrive/Shared drives/CIS 530 Project/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/Shared drives/CIS 530 Project


# Embeddings from Transformer models

##BERT

In [4]:
def load_obj(name ):
  with open('obj/' + name + '.pkl', 'rb') as f:
    return pickle.load(f)

# Returns a tuple (numpy matrix, word_to_index dict) where the matrix contains all the 
# X-dimensional word embeddings, and word_to_index is a dictionary from the
# word into the index in the matrix. For Out-of-vocabulary words it uses
# a [UNK] or <unused> embedding
def get_indexed_mbert_word_embeddings():
  loaded_list = load_obj('mbert_matrix_v2')
  loaded_np_matrix = np.asarray(loaded_list)
  loaded_word_to_idx = load_obj('mbert_word_to_index_v2')
  return loaded_np_matrix, loaded_word_to_idx 

mat, idx = get_indexed_mbert_word_embeddings()
# embed = mat[idx.get('cat')]
print(len(idx))
# print(type(embed))
# print(embed.shape)
print(mat.shape)


69439
(69438, 768)


## BETO

In [5]:
!pip3 install transformers
# !wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
# !wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
# !wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
# !tar -xzvf pytorch_weights.tar.gz
# !mv config.json pytorch/.
# !mv vocab.txt pytorch/.

from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModelWithLMHead.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model.eval()
model.cuda()

# Returns a tuple (numpy matrix, word_to_index dict) where the matrix contains all the 
# X-dimensional word embeddings, and word_to_index is a dictionary from the
# word into the index in the matrix. For Out-of-vocabulary words it uses
# a [UNK] or <unused> embedding
def get_indexed_beto_word_embeddings():
  path = "pytorch/vocab.txt"
  idx=0
  word_to_index = dict()
  with open(path, 'r') as f:
    for line in f:
        line = line.rstrip('\n').split('\t')
        word_to_index[line[0]] = idx
        idx=idx+1 

  matrix = model.get_input_embeddings().weight.data
  np_matrix = matrix.cpu().numpy()

  return np_matrix, word_to_index

# mat, word_to_idx = get_indexed_beto_word_embeddings()
# print(mat.shape)
# embed = mat[word_to_idx.get('para'),3] #default value if word not found is <UNK> or index 3.
# print(embed.shape)
# print(type(embed))
# print(embed)

     |████████████████████████████████| 573kB 4.7MB/s 
     |████████████████████████████████| 1.0MB 64.5MB/s 
     |████████████████████████████████| 3.7MB 51.0MB/s 
     |████████████████████████████████| 890kB 65.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=9568430559e503ee0c1786ee6fff50a4e2ea74b5752306e2eda52cbf00c558d7
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


# Utility Functions

In [0]:
# Returns the dictionary from the file with translations
def english_to_spanish_dict(path):
  en_to_spanish = dict()

  with open(path, 'r') as f:
    for line in f:
        line = line.rstrip('\n').split('\t')
        en_word = line[0]
        es_word = line[1]
        if en_word.isupper(): en_to_spanish[en_word] = es_word.upper()
        elif len(en_word) > 0 and en_word[0].isupper(): en_to_spanish[en_word] = es_word.capitalize()
        else: en_to_spanish[en_word] = es_word

  return en_to_spanish

# Returns a tuple (matrix, word_to_index) where the matrix contains all the 
# X-dimensional word embeddings, and word_to_index is a dictionary from the
# word into the index in the matrix. For Out-of-vocabulary words it creates
# a random embedding
def get_indexed_word_embeddings(translation_path, embedding_path):
  en_to_spanish = english_to_spanish_dict(translation_path)
  vectors = Magnitude(embedding_path)

  dim = vectors.dim
  
  matrix = [] # words by embedding-dimension
  word_to_index = dict()

  index = 0

  en_files = ['Data/eng.train', 'Data/eng.testa', 'Data/eng.testb']
  
  for file in en_files:
    with open(file, 'r') as f:
      for line in f:
        line = line.rstrip('\n')
        if line == "": continue

        word = line.split()[0]
        trans = en_to_spanish[word]
        if trans not in word_to_index:
          word_to_index[trans] = index
          if trans in vectors: matrix.append(vectors.query(trans))
          else: matrix.append(np.random.uniform(-(3/dim)**0.5, (3/dim)**0.5, dim))

          index += 1

  es_files = ['Data/esp.train', 'Data/esp.testa', 'Data/esp.testb']
  
  for file in es_files:
    with open(file, 'r') as f:
      for line in f:
        line = line.rstrip('\n')
        if line == "": continue

        word = line.split()[0]
        if word not in word_to_index:
          word_to_index[word] = index
          if word in vectors: matrix.append(vectors.query(word))
          else: matrix.append(np.random.uniform(-(3/dim)**0.5, (3/dim)**0.5, dim))
            
          index += 1

  return matrix, word_to_index

#returns char dictionary created from all path in paths
def create_char_index(paths, es_translation_dict, pad=False): #AS IS
    char_dict = {}
    if pad:
        char_dict[PAD_WORD] = PAD
        char_dict[UNK_WORD] = UNK
    else:
        char_dict[UNK_WORD] = 0

    for path in paths:
        for line in open(path):
            l = line.strip().split()
            if len(l) > 0:# and l[0] != '':
              #l[0] is word l[1] is POS, l[2] is gold standard NER label
              word = l[0]
              es_word = word
              if word in es_translation_dict:
                es_word = es_translation_dict.get(word)
              for i in range(len(es_word)):
                  if es_word[i] not in char_dict:
                      char_dict[es_word[i]] = len(char_dict)

    return char_dict

#   Returns 
#1. all the spanish 'sentences' in 2D array
#2. 2D array indicating if word in given sentence is OOV word (True if the word is used as-is, translation not found) or not
#3. 2D array returning labels.
#   in the file(s) at paths in an array.
def data_to_words_sentences(paths, es_translation_dict, test=False):
  sentences=[]
  curr_sentence=[]
  OOV = []
  labels=[]
  curr_OOV_sentence = []
  curr_label_sentence = []
  word_idx = 0
  for path in paths:
      for line in open(path):
        line = line.strip().split()
        
        end_of_line=False
        if len(line) == 0:
          end_of_line=True
        
        if not end_of_line:
          word = line[0]
          es_word = word

          if not test: #english has 4th column label
            curr_label_sentence.append(line[3])
          else: #spanish has 3rd column label
            curr_label_sentence.append(line[2])

          curr_OOV_sentence.append(True)
          if test:
            curr_OOV_sentence[word_idx] = False
          else: #not test
            if word in es_translation_dict:
              es_word = es_translation_dict.get(word)
              curr_OOV_sentence[word_idx] = False
          
          word_idx = word_idx+1
          curr_sentence.append(es_word)
        if end_of_line:
          sentences.append(curr_sentence)
          OOV.append(curr_OOV_sentence)
          labels.append(curr_label_sentence)
          curr_sentence=[]
          curr_OOV_sentence=[]
          curr_label_sentence=[]
          word_idx=0
  return sentences, OOV, labels



##Get_input fucntion

In [0]:
#ADD all methods to get initial input to neural network here. Returns char, labels and word input
#OOV[i] indicates if word[i] is OOV (translation not found)
def get_input(transformer_word_vocab_dict, word_vocab_dict, sentences, OOV, char_vocab_dict, token_labels, label_to_index):
  max_word_len = max(len(word) for sentence in sentences for word in sentence)
  #max_word_len = max(len(word) for word in words)
  max_sentence_length = max(len(sentence) for sentence in sentences)

  word_input = np.zeros((len(sentences), max_sentence_length), dtype='int64')
  word_input_length = [len(sentence) for sentence in sentences]

  char_input = np.zeros((len(sentences), max_sentence_length,max_word_len), dtype='int64') 
  char_input_length = np.zeros((len(sentences), max_sentence_length), dtype='int64') #2D array of length of word in each sentence in sentences
  
  # word_mbert_input = get_mbert_es_embeddings(sentences, max_sentence_length)
  transformer_word_input = np.zeros((len(sentences), max_sentence_length), dtype='int64')


  label_input = np.zeros((len(sentences), max_sentence_length), dtype='int64') #2D array of label of word in each sentence in sentences
  for i in range(len(sentences)):
    for j in range(len(sentences[i])):
      word_input[i][j] = word_vocab_dict[sentences[i][j]]
      transformer_word_input[i][j] = transformer_word_vocab_dict[sentences[i][j]]

      char_input_length[i][j] = len(sentences[i][j]) 
      label_input[i][j] = label_to_index.index(token_labels[i][j])

      for k in range(len(sentences[i][j])):
        c = sentences[i][j][k]
        if c in char_vocab_dict:
          input_zero = c.isdigit() or OOV[i][j]
          char_input[i][j][k] = char_vocab_dict['0' if input_zero else c]
        else:
          char_input[i][j][k] = UNK

  word_input_var = Variable(torch.from_numpy(word_input), requires_grad=False)
  transformer_word_input_var = Variable(torch.from_numpy(transformer_word_input), requires_grad=False)
  word_input_length_var = Variable(torch.LongTensor(word_input_length), requires_grad=False)
  label_var = Variable(torch.from_numpy(label_input), requires_grad=False)
  char_input_var = Variable(torch.from_numpy(char_input), requires_grad=False)
  char_input_length_var = Variable(torch.from_numpy(char_input_length), requires_grad=False)
  return transformer_word_input_var.cuda(), word_input_var.cuda(), word_input_length_var.cuda(), char_input_var.cuda(), char_input_length_var.cuda(), label_var.cuda()

def batch_from_data(X, X1, y, batch_size, random=True):
    batch_num = int(np.ceil(len(y) / float(batch_size)))
    rand_indices = np.arange(len(y))
    if random: rand_indices = np.random.permutation(len(y))

    for batch in range(0, batch_num):
        bs = batch_size if batch < batch_num - 1 else len(y) - batch_size * batch
        #from pdb import set_trace as debug
        #debug()
        yield [X[i] for i in rand_indices[batch * batch_size : batch * batch_size + bs]], [X1[i] for i in rand_indices[batch * batch_size : batch * batch_size + bs]], [y[i] for i in rand_indices[batch * batch_size : batch * batch_size + bs]]

# Model

##char embedding

In [0]:
class char_model(nn.Module):
    def __init__(self, char_vocab_size, char_embed_size, char_lstm_hidden_size=50):

        #START: char embedding section
        super(char_model, self).__init__()
        self.char_embed = nn.Embedding(char_vocab_size, char_embed_size, padding_idx=PAD)
        self.char_lstm = nn.LSTM(char_embed_size, char_lstm_hidden_size, bidirectional=True, batch_first=True)
        self.char_lstm_hidden_size = char_lstm_hidden_size
        #END: char embedding section 

    def forward(self, char_inp, char_input_length):

        #START: char embedding section
        #from pdb import set_trace as debug
        #debug()
        char_input = char_inp.view(-1, char_inp.size(2))
        char_input_length_sorted, char_original_idx = char_input_length.view(-1).sort(0, descending=True)
        char_embedded = self.char_embed(char_input)
        char_embedded_sorted = char_embedded[char_original_idx] #get embeddings in descending order of length of word in words

        char_input_length_sorted_size = char_input_length_sorted.size(0)
        last_index = char_input_length_sorted_size
        if char_input_length_sorted.data.eq(0).sum() != 0: #atleast 1 element of char_input_length_sorted is/are zero
          last_index = char_input_length_sorted.data.eq(0).nonzero()[0][0]
        char_embedded_sorted = char_embedded_sorted[:last_index]
        char_input_length_sorted = char_input_length_sorted[:last_index]

        char_input_packed_padded = pack_padded_sequence(char_embedded_sorted, char_input_length_sorted.cpu().data.numpy(), batch_first=True)
        char_output_packed_padded, (h_n, c_n) = self.char_lstm(char_input_packed_padded)
        char_hidden_state = torch.cat([h_n[0], h_n[1]], 1)

        if last_index != char_input_length_sorted_size:
          zero_padding_diff = char_input_length_sorted_size - last_index
          zero_padding = Variable(torch.zeros((zero_padding_diff, 2*self.char_lstm_hidden_size)), requires_grad=False).cuda()
          char_hidden_state = torch.cat([char_hidden_state, zero_padding], 0)

        char_hidden_state = char_hidden_state[torch.argsort(char_original_idx)] #char_hidden_state[torch.from_numpy(np.argsort(char_original_idx.cpu().data.numpy())).cuda()]
        char_hidden_state = char_hidden_state.view(char_inp.size(0), -1, char_hidden_state.size(1))
        return char_hidden_state
        #END: char embedding section 

    def reset_parameters(self):

      #START: char embedding section
      for param in self.char_embed.parameters():
          nn.init.normal(param, mean=0, std=0.01)

      for name, param in self.char_lstm.named_parameters():
          if 'bias' in name:
              nn.init.constant_(param, 0.)
          elif 'weight' in name:
              nn.init.normal(param, mean=0, std=0.1)
      #END: char embedding section 

##Self Attention

In [0]:
class selfAttention(nn.Module):

  def __init__(self, transformer_hidden_size, transformer_matrix, char_lstm_hidden_size, word_vocab_size, word_embedding_size, word_lstm_hiddden_size, word_vector):
    super(selfAttention, self).__init__()
    self.word_embed = nn.Embedding(word_vocab_size, word_embedding_size, padding_idx=PAD)
    self.transformer_matrix = transformer_matrix
    self.transformer_hidden_size = transformer_hidden_size

    self.word_lstm = nn.LSTM(2*char_lstm_hidden_size + word_embedding_size + transformer_hidden_size, word_lstm_hiddden_size, batch_first=True, bidirectional=True)
    self.word_linear = nn.Linear(word_lstm_hiddden_size * 2, word_lstm_hiddden_size * 2)

    self.tanh = nn.Tanh()
    self.softmax = nn.Softmax(dim=2)
    self.embedding_dropout = nn.Dropout(0.5)
    self.word_dropout = nn.Dropout(0.5)
    self.att_sm_dropout = nn.Dropout(0.5)
    #self.att_dropout= nn.Dropout(0.2)
    self.word_embed.weight.data.copy_(torch.from_numpy(np.asarray(word_vector)))


  def forward(self, transformer_word_input, words, char_hidden_state, word_length):
    # word_embedding = self.word_embed(words).cuda()
    word_embedding = self.word_embed(words)

    transformer_word_embed = (self.TransformerEmbedder(transformer_word_input)).float().cuda()
    # debug()
    word_lstm_input = torch.cat([transformer_word_embed, word_embedding, char_hidden_state], 2) #here

    word_lstm_input = self.embedding_dropout(word_lstm_input)
    
    word_length, word_idx = word_length.sort(0, descending=True)
    word_lstm_input = word_lstm_input[word_idx]

    word_packed_input = pack_padded_sequence(word_lstm_input, word_length.cpu().data.numpy(), batch_first=True)
    word_packed_output, _ = self.word_lstm(word_packed_input)
    word_output, _ = pad_packed_sequence(word_packed_output, batch_first=True)
    word_output = word_output[torch.from_numpy(np.argsort(word_idx.cpu().data.numpy())).cuda()]
    
    word_output = self.word_dropout(word_output)
    attn_input = self.tanh(self.word_linear(word_output))

    att_padding_mask = Variable(words.data.ne(PAD)).cuda()
    context = attn_input * att_padding_mask.float().unsqueeze(2)
    attn_out = context.bmm(context.transpose(1, 2))

    attention_self_mask = Variable(1 - torch.eye(words.size(1), words.size(1))).cuda()
    attn_out = attn_out * attention_self_mask.unsqueeze(0)

    out = self.softmax(attn_out)
    out = out * att_padding_mask.float().unsqueeze(2)
    out = out * att_padding_mask.float().unsqueeze(1)
    out = self.att_sm_dropout(out)
    context_v = out.bmm(word_output)
    #context_v = self.att_dropout(context_v)
    word_output = torch.cat([word_output, context_v], 2)

    return word_output


  def TransformerEmbedder(self, transformer_word_input):
    b, s = transformer_word_input.shape # batch, sentence
    h = self.transformer_hidden_size # hidden
    batch_embed = np.zeros((b, s, h))
    # debug()
    for i, sentence_as_idx in enumerate(transformer_word_input): # indexing vectorized O(n)
      sentence_embed = self.transformer_matrix[sentence_as_idx.cpu()] # sentence_length x embed_hidden_size
      batch_embed[i, :, :] = sentence_embed
    return torch.from_numpy(batch_embed)


  def reset_parameters(self):

    for name, param in self.word_lstm.named_parameters():
        if 'bias' in name:
            nn.init.constant(param, 0.)
        elif 'weight' in name:
            nn.init.normal(param, mean=0, std=0.1)

## CRF

In [0]:
def logsumexp(x, dim=None): #AS IS
    if dim is None:
        xmax = x.max()
        xmax_ = x.max()
        return xmax_ + torch.log(torch.exp(x - xmax).sum())
    else:
        xmax, _ = x.max(dim, keepdim=True)
        xmax_, _ = x.max(dim)
        return xmax_ + torch.log(torch.exp(x - xmax).sum(dim))


In [0]:
class CRF_Module(nn.Module):
    def __init__(self, input_size, num_labels, bigram=True):


        super(CRF_Module, self).__init__()
        self.pad_label_id = num_labels
        self.bigram = bigram
        self.input_size = input_size
        self.num_labels = num_labels + 1
        self.state_layer = nn.Linear(input_size, self.num_labels)

        if bigram: # 
            self.transition_layer = nn.Linear(input_size, self.num_labels * self.num_labels) # transition weights are learned (costs of moving from one tag to next)
            self.register_parameter('transition_matrix', None)
        else:
            self.transition_layer = None
            self.transition_matrix = Parameter(torch.Tensor(self.num_labels, self.num_labels)) # initialize a transition matrix instead 

        self.reset_parameters()

    def forward(self, input, mask=None):
      batch, length, _ = input.size()
      out_state = self.state_layer(input).unsqueeze(2)

      if self.bigram:
          out_transition = self.transition_layer(input).view(batch, length, self.num_labels, self.num_labels)
          net_output = out_transition + out_state
      else:
          net_output = self.transition_matrix + out_state

      if mask is not None:
          net_output = net_output * mask.unsqueeze(2).unsqueeze(3)
      return net_output


    def reset_parameters(self):
      nn.init.constant(self.state_layer.bias, 0.)
      if self.bigram:
          nn.init.xavier_uniform(self.transition_layer.weight)
          nn.init.constant(self.transition_layer.bias, 0.)
      else:
          nn.init.normal(self.transition_matrix)


    def _viterbi_decode(self, input, mask, leading_symbolic=0):
      energy = self.forward(input, mask=mask).data
      energyTrans = energy.transpose(0, 1) #energy_transpose
      energyTrans = energyTrans[:, :, leading_symbolic:-1, leading_symbolic:-1]

      w_len, batch_size, num_label, _ = energyTrans.size()
      batch_index = torch.arange(0, batch_size).long().cuda()
      
      curr_mat = torch.zeros([w_len, batch_size, num_label, 1]).cuda()
      pointer = torch.cuda.LongTensor(w_len, batch_size, num_label).zero_()
      back_pointer = torch.cuda.LongTensor(w_len, batch_size).zero_()

      curr_mat[0] = energy[:, 0, -1, leading_symbolic:-1].unsqueeze(2)
      pointer[0] = -1
      for t in range(1, w_len):
          prev_mat = curr_mat[t - 1]
          temp_mat, pointer[t] = torch.max(energyTrans[t] + prev_mat, dim=1)
          curr_mat[t] = temp_mat.unsqueeze(2)

      _, back_pointer[-1] = torch.max(curr_mat[-1].squeeze(2), dim=1)
      for t in reversed(range(w_len - 1)):
          pointer_last = pointer[t + 1]
          back_pointer[t] = pointer_last[batch_index, back_pointer[t + 1]]

      return back_pointer.transpose(0, 1) + leading_symbolic

    
    def loss(self, input, target, mask=None):
      #debug()
      batch, length, _ = input.size()
      energy = self.forward(input, mask=mask)
      energy_transpose = energy.transpose(0, 1)
      target_transpose = target.transpose(0, 1)
      mask_transpose = None
      if mask is not None:
          mask_transpose = mask.unsqueeze(2).transpose(0, 1)

      partition = None

      batch_index = torch.arange(0, batch).long().cuda()
      prev_label = torch.cuda.LongTensor(batch).fill_(self.num_labels - 1)
      tgt_energy = Variable(torch.zeros(batch)).cuda()

      for t in range(length):
          curr_energy = energy_transpose[t]
          if t == 0:
              partition = curr_energy[:, -1, :]
          else:
              partition_new = logsumexp(curr_energy + partition.unsqueeze(2), dim=1)
              if mask_transpose is None:
                  partition = partition_new
              else:
                  mask_t = mask_transpose[t]
                  partition = partition + (partition_new - partition) * mask_t
          tgt_energy += curr_energy[batch_index, prev_label, target_transpose[t].data]
          prev_label = target_transpose[t].data

      return logsumexp(partition, dim=1) - tgt_energy



## wrapper code

In [0]:
class Attention_LSTM_CRF(nn.Module):
    def __init__(self, transformer_hidden_size, transformer_matrix, char_vocab_size, char_embed_size, word_vocab_size, word_embedding_size, word_lstm_hiddden_size, word_vector, num_labels, bigram=True, char_lstm_hidden_size=50):
        super(Attention_LSTM_CRF, self).__init__()
        self.char_vocab_size = char_vocab_size
        self.char_embed_size = char_embed_size
        self.word_vocab_size = word_vocab_size
        self.word_embedding_size = word_embedding_size
        self.word_lstm_hiddden_size = word_lstm_hiddden_size
        self.char_lstm_hidden_size = char_lstm_hidden_size
        self.word_vector = word_vector
        self.transformer_matrix = transformer_matrix
        self.transformer_hidden_size = transformer_hidden_size

        print("from hinside", self.transformer_hidden_size)

        # crf vars
        self.num_labels = num_labels
        self.bigram = bigram

        self.charModel = char_model(self.char_vocab_size, self.char_embed_size, self.char_lstm_hidden_size)
        self.Attention = selfAttention(self.transformer_hidden_size, self.transformer_matrix, self.char_lstm_hidden_size, self.word_vocab_size, self.word_embedding_size, self.word_lstm_hiddden_size, self.word_vector)
        self.CRF = CRF_Module(self.word_lstm_hiddden_size*4, num_labels)
    
    def forward(self, transfer_word_input, words, input, word_length, char_input_length, target, hidden=None):
        charOut = self.charModel(input, char_input_length) #basically char_hidden_state
        AttentionOut = self.Attention(transfer_word_input, words, charOut, word_length)
        CRFLossOut = self.CRF.loss(AttentionOut, target, words.ne(PAD).float()).mean()
        CRFPredict = self.CRF._viterbi_decode(AttentionOut, words.ne(PAD).float(), 1)

        return CRFLossOut, CRFPredict


##Training and Validation

In [0]:
def training(model_save_file, trans_word_vocab_dict, es_translation_dict , char_vocab_dict, model, optimizer, lr, epochs):
  best_f1_score = 0.0
  
  all_train_loss = []
  all_val_loss = []
  all_train_acc = []
  all_val_acc = []
  all_train_f1 = []
  all_val_f1 = []

  for epoch in range(epochs):
      labels_global = []
      pred_global = []

      model.train()

      epoch_loss = 0
      total = 0
      correct = 0
      batch =0
      
      
      #es_translation_dict = english_to_spanish_dict("translation_bi.txt")      
      train_paths = ['Data/eng.train', 'Data/eng.testa', 'Data/eng.testb']
      #char_vocab_dict = create_char_index(train_paths, es_translation_dict, pad=False)
      sentences, OOV, labels = data_to_words_sentences(train_paths, es_translation_dict, test=False) #whole data
      #train_X = [words, OOV]
      
      for data in batch_from_data(sentences, OOV, labels, 16):

          bsentences, bOOV, y = data
          #label_input
          
          #[bsentences, bOOV] = X
          transformer_word_input, word_input, word_length_input, char_input, char_length_input, label_input = get_input(trans_word_vocab_dict, word_to_index, bsentences, bOOV, char_vocab_dict, y, LABEL_INDEX)
          optimizer.zero_grad()
          true_labels = label_input.contiguous().view(-1)

          #forward
          loss, predict = model(transformer_word_input, word_input, char_input, word_length_input, char_length_input, label_input)
          predict = predict.contiguous().view(-1)

          total +=  true_labels.data.ne(PAD).float().sum()
          pred_correct = predict.eq(true_labels.data).masked_select(true_labels.ne(PAD).data).float().sum()

          loss.backward()
          torch.nn.utils.clip_grad_norm(model.parameters(), 5)
          optimizer.step()

          epoch_loss += loss.item()
          correct += pred_correct
          batch+=1

          labels_global.extend(true_labels.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())
          pred_global.extend(predict.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())
          
          # true_labels_list = list(true_labels.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())
          # predict = list(predict.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())
          # bsentences = [j for i in bsentences for j in i]
          # for i in range(len(bsentences)):
          #   print(bsentences[i] + "\t" + LABEL_INDEX[true_labels_list[i]] + "\t" + LABEL_INDEX[predict[i]])

          if batch%200==0:
            print("Batch {} loss: {:.2f}".format(batch, epoch_loss/batch))
    
      f1 = f1_score(labels_global, pred_global, average='macro')
      
      print("Epoch {} training loss: {:.4f}, training accuracy: {:.4f}, f1 score {:.2f}".format(epoch, epoch_loss/batch, correct * 100.0/total, f1))

      all_train_loss.append(epoch_loss/batch)
      all_train_acc.append(correct * 100.0/total)
      all_train_f1.append(f1)

      lr = lr / (1.0 + epoch * 0.05) #decay=0.05
      for param_group in optimizer.param_groups:
          param_group['lr'] = lr
      val_f1, val_loss, val_acc = evaluate(trans_word_vocab_dict, es_translation_dict , char_vocab_dict, model) # changed signature

      all_val_f1.append(val_f1)
      all_val_acc.append(val_acc)
      all_val_loss.append(val_loss)

      # store at best f1
      if val_f1 > best_f1_score :
        best_f1_score = val_f1
        print("Saving model on best val F1 so far " + str(val_f1))
        torch.save(model.state_dict(), model_save_file)

  return  all_train_loss, all_val_loss , all_train_acc, all_val_acc , all_train_f1 , all_val_f1


def evaluate(trans_word_vocab_dict, es_translation_dict , char_vocab_dict, model):

    model.eval()

    correct = 0
    total = 0
    test_loss = 0
    batch = 0

    labels_global = []
    pred_global = []

    #es_translation_dict = english_to_spanish_dict("translation_bi.txt")      
    val_paths = ['Data/esp.testa']
    #char_vocab_dict = create_char_index(val_paths, es_translation_dict, pad=False)
    sentences, OOV, labels = data_to_words_sentences(val_paths, es_translation_dict, test=True) #whole data

    #val_X = [words, OOV]
    for data in batch_from_data(sentences, OOV, labels, 16):
        
        bsentences, bOOV, y = data
        #label_input
        
        #[bsentences, bOOV] = X
        
        transformer_word_input, word_input, word_length_input, char_input, char_length_input, label_input = get_input(trans_word_vocab_dict, word_to_index, bsentences, bOOV, char_vocab_dict, y, LABEL_INDEX)
        loss, predict = model(transformer_word_input, word_input, char_input, word_length_input, char_length_input, label_input)
        predict = predict.contiguous().view(-1)
        test_loss += loss.item()


        true_labels = label_input.contiguous().view(-1)
        total += true_labels.data.ne(PAD).float().sum()
        pred_correct = predict.eq(true_labels.data).masked_select(true_labels.ne(PAD).data).float().sum()
        correct += pred_correct
        batch+=1

        labels_global.extend(true_labels.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())
        pred_global.extend(predict.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())

        # true_labels_list = list(true_labels.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())
        # predict = list(predict.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())
        # bsentences = [j for i in bsentences for j in i]
        # for i in range(len(bsentences)):
        #   print(bsentences[i] + "\t" + LABEL_INDEX[true_labels_list[i]] + "\t" + LABEL_INDEX[predict[i]])

    test_acc = correct * 100.0 / total

    f1 = f1_score(labels_global, pred_global, average='macro')

    print("loss: {:.4f} eval acc: {:.4f} | f1 {:.4f}".format(test_loss/batch, test_acc, f1))
    return f1, test_loss/batch, test_acc

def write_to_results(filename, trans_word_vocab_dict, es_translation_dict , char_vocab_dict, model):

    model.eval()

    labels_global = []
    pred_global = []
   
    test_paths = ['Data/esp.train', 'Data/esp.testb']
    sentences, OOV, labels = data_to_words_sentences(test_paths, es_translation_dict, test=True) #whole data

    for data in batch_from_data(sentences, OOV, labels, 16, random=False):
        # debug()
        bsentences, bOOV, y = data
        
        transformer_word_input, word_input, word_length_input, char_input, char_length_input, label_input = get_input(trans_word_vocab_dict, word_to_index, bsentences, bOOV, char_vocab_dict, y, LABEL_INDEX)
        loss, predict = model(transformer_word_input, word_input, char_input, word_length_input, char_length_input, label_input)
        predict = predict.contiguous().view(-1)

        true_labels = label_input.contiguous().view(-1)

        labels_global.extend(list(true_labels.masked_select(true_labels.ne(PAD).data).cpu().data.numpy()))
        pred_global.extend(list(predict.masked_select(true_labels.ne(PAD).data).cpu().data.numpy()))
        # true_labels_list = list(true_labels.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())
        # predict = list(predict.masked_select(true_labels.ne(PAD).data).cpu().data.numpy())
        # bsentences = [j for i in bsentences for j in i]
        # for i in range(len(bsentences)):
        #   print(bsentences[i] + "\t" + LABEL_INDEX[true_labels_list[i]] + "\t" + LABEL_INDEX[predict[i]])

    sentence_tokens = [j for i in sentences for j in i]

    with open(filename, "w") as f:
        for i in range(len(sentence_tokens)):
            f.write(sentence_tokens[i] + "\t" + LABEL_INDEX[labels_global[i]] + "\t" + LABEL_INDEX[pred_global[i]] + "\n")


#Training and Evaluating

##Loading data

In [14]:
cd /content/gdrive/Shared drives/CIS 530 Project

/content/gdrive/Shared drives/CIS 530 Project


In [0]:
train_paths = ['Data/eng.train', 'Data/eng.testa', 'Data/eng.testb']
val_paths = ['Data/esp.testa']
test_paths = ['Data/esp.train', 'Data/esp.testb']


es_translation_dict = english_to_spanish_dict("translations_bi.txt") 
all_paths=[]
all_paths.extend(train_paths)
all_paths.extend(val_paths)
all_paths.extend(test_paths)  
char_vocab_dict = create_char_index(all_paths, es_translation_dict, pad=False)
matrix, word_to_index = get_indexed_word_embeddings("translations_bi.txt", "spanish.glove.gigaword_wiki.100d.magnitude") #word_vector, word_

## Training

In [17]:
# change model name, generate results below
global model_save_file 
model_save_file="Mbert_Extension_decay1e4" 
global results_save_file
results_save_file = "results_Mbert_Extension_decay1e4.txt"

transformer_matrix, trans_word_vocab_dict = get_indexed_mbert_word_embeddings()
transformer_hidden_size = transformer_matrix.shape[1]
# transformer_matrix_tensor = torch.from_numpy(transformer_matrix)

print(transformer_hidden_size)
# model init call
model = Attention_LSTM_CRF(transformer_hidden_size, transformer_matrix, len(char_vocab_dict), 25, len(matrix), 100, 200, matrix, len(LABEL_INDEX)).cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.015, momentum=0.9, weight_decay=1e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0)


all_train_loss, all_val_loss , all_train_acc, all_val_acc , all_train_f1 , all_val_f1 = training(model_save_file, trans_word_vocab_dict, es_translation_dict, char_vocab_dict, model, optimizer, 0.015, 30)


768
from hinside 768


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


##saving model

## Open model and write to results

In [18]:

model = Attention_LSTM_CRF(transformer_hidden_size, transformer_matrix, len(char_vocab_dict), 25, len(matrix), 100, 200, matrix, len(LABEL_INDEX)).cuda()
model.load_state_dict(torch.load(model_save_file))

write_to_results(results_save_file, trans_word_vocab_dict, es_translation_dict, char_vocab_dict, model)

from hinside 768


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
